In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import requests
from tqdm import tqdm
import time
import os
import re
import datetime
import numpy as np

In [2]:
pd.options.display.max_columns = 150
nanValue = np.nan

In [3]:
def getTables(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    html_string = str(bs)
    html_io = StringIO(html_string)
    tables = pd.read_html(html_io)
    df = tables
    return bs, df

In [4]:
#id = 202406030811#皐月賞
#id = 202404010801#新人ジョッキー
#id = 202305050812#ジャパンカップ
#id = 201005030211#1着同着
id = 202209050411#2着同着
url = 'https://db.netkeiba.com/race/' +str(id)
url2 = 'https://race.netkeiba.com/race/result.html?race_id=' + str(id) + '&rf=race_list'

In [5]:

colName = ['raceId', '着順', '枠番', '馬番', '馬名', '馬ID', '性別','年齢', '斤量', '騎手','騎手ID', 'タイム', '着差', '人気', '単勝オッズ',\
           '第一コーナー', '第二コーナー', '第三コーナー', '第四コーナー', '東西', '調教師', '調教師ID', '馬体重', '馬体重(前走)', '増減']
dataFrame = pd.DataFrame(columns=colName)
print(dataFrame.columns)
bs, df = getTables(url)
df = df[0]
time.sleep(1)
_, df2 = getTables(url2)
df2 = df2[0]
prehourse_goal_time =0
nowhourse_goal_time = 0

pattern = r'[▲△☆★]'
testFrame = pd.DataFrame(columns=colName)
for i in range(0, len(df)):
    new_row = {}
    new_row['raceId'] = '202405020801'#後で変更
    new_row['着順'] = df['着 順'][i]
    new_row['枠番'] = df['枠 番'][i]
    new_row['馬番'] = df['馬 番'][i]
    new_row['馬名'] = df['馬名'][i]
    horseName = df['馬名'][i]
    link = bs.find('a', {'title': horseName})['href']
    horse_id = link.split('/')[-2]
    new_row['馬ID'] = horse_id
    gender = df['性齢'].map(lambda x: str(x)[0])
    age = df['性齢'].map(lambda x: str(x)[1:])
    new_row['性別'] = gender[i]
    new_row['年齢'] = age[i]
    new_row['斤量'] = df['斤量'][i]
    
    jockeyName = df['騎手'][i]
    link = bs.find('a', {'title': jockeyName})['href']
    jockey_id = link.split('/')[-2]
    cleaned_jockey_name = re.sub(pattern, '', df['騎手'][i])
    new_row['騎手'] = cleaned_jockey_name
    #騎手にマークが付いていることがある

    new_row['騎手ID'] = jockey_id
    
    if df['着 順'][i] == '除' or df['着 順'][i] == '中':
        new_row['タイム'] = nanValue
        new_row['着差'] = nanValue
        new_row['人気'] = nanValue
        new_row['単勝オッズ'] = nanValue
    else:
        goaltime_min = df['タイム'].map(lambda x: str(x)[0])
        goaltime_min = int(goaltime_min[i])*60
        goaltime_second = df['タイム'].map(lambda x: str(x)[2:])
        goaltime_second = float(goaltime_second[i])
        nowhourse_goal_time = goaltime_min + goaltime_second
        new_row['タイム'] = nowhourse_goal_time
        new_row['着差'] = round(nowhourse_goal_time - prehourse_goal_time,1) if prehourse_goal_time != 0 else 0
        prehourse_goal_time = nowhourse_goal_time
        new_row['人気'] = df['人 気'][i]
        new_row['単勝オッズ'] = float(df['単勝'][i])
    corner = str(df2['コーナー 通過順'][i]).split('-')
    new_row['第一コーナー'] = corner[-4] if len(corner) >= 4 else ''
    new_row['第二コーナー'] = corner[-3] if len(corner) >= 3 else ''
    new_row['第三コーナー'] = corner[-2] if len(corner) >= 2 else ''
    new_row['第四コーナー'] = corner[-1] if len(corner) >= 1 else ''
    trainer_location = df['調教師'][i][:3]
    if '[東]' in trainer_location:
        new_row['東西'] = '東'
    elif '[西]' in trainer_location:
        new_row['東西'] = '西'
    elif '[地]' in trainer_location:
        new_row['東西'] = '地'
    else:
        new_row['東西'] = '外'
    
    #new_row['東西'] = df['調教師'][i][:3].replace('[','').replace(']','')
    new_row['調教師'] = df['調教師'][i][4:]
    trainerName = df['調教師'][i][4:]
    link = bs.find('a', {'title': trainerName})['href']
    trainer_id = link.split('/')[-2]
    new_row['調教師ID'] = trainer_id
    hourseWeight =  int(df['馬体重'][i][:3])
    new_row['馬体重'] = hourseWeight
    weight_change = df['馬体重'][i][3:].replace('(','').replace(')','')
    weight_change = int(weight_change)
    new_row['馬体重(前走)'] = hourseWeight - weight_change
    new_row['増減'] = weight_change
    for clna in ['着順','枠番','馬番','馬ID','年齢', '騎手ID','人気','第一コーナー', '第二コーナー', '第三コーナー', '第四コーナー','馬体重' ,'調教師ID','馬体重(前走)','増減']:
        new_row[clna] = pd.to_numeric(new_row[clna], errors='coerce')
    for clna in ['単勝オッズ','タイム', '着差','斤量']:
        new_row[clna] = pd.to_numeric(new_row[clna], downcast='float')
    #後で修正
    testFrame = pd.concat((testFrame,pd.DataFrame([new_row])), ignore_index=True)
print(testFrame)
testFrame.to_pickle('test.pkl')

Index(['raceId', '着順', '枠番', '馬番', '馬名', '馬ID', '性別', '年齢', '斤量', '騎手', '騎手ID',
       'タイム', '着差', '人気', '単勝オッズ', '第一コーナー', '第二コーナー', '第三コーナー', '第四コーナー',
       '東西', '調教師', '調教師ID', '馬体重', '馬体重(前走)', '増減'],
      dtype='object')
          raceId  着順 枠番  馬番         馬名         馬ID 性別 年齢  斤量    騎手  騎手ID  \
0   202405020801   1  8  18   ジェラルディーナ  2018105081  牝  4  56  Ｃ．デム  5473   
1   202405020801   2  7  13    ウインマリリン  2017101431  牝  5  56   レーン  5585   
2   202405020801   2  7  15      ライラック  2019103588  牝  3  54  Ｍ．デム  5212   
3   202405020801   4  7  14      アカイイト  2017106203  牝  5  56   幸英明   732   
4   202405020801   5  6  11      ナミュール  2019105213  牝  3  54  横山武史  1170   
5   202405020801   6  2   4   デアリングタクト  2017100720  牝  5  56  松山弘平  1126   
6   202405020801   7  3   6   ホウオウエミーズ  2017104691  牝  5  56  丸田恭介  1117   
7   202405020801   8  6  12  ルビーカサブランカ  2017105582  牝  5  56  横山和生  1140   
8   202405020801   9  2   3       ピンハイ  2019102772  牝  3  54  川田将雅  1088   
9   20240

C:\Users\shirase\AppData\Local\Temp\ipykernel_13140\236888744.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testFrame = pd.concat((testFrame,pd.DataFrame([new_row])), ignore_index=True)


In [6]:
raceResultDf = pd.read_pickle('test.pkl')
raceResultDf

,raceId,着順,枠番,馬番,馬名,馬ID,性別,年齢,斤量,騎手,騎手ID,タイム,着差,人気,単勝オッズ,第一コーナー,第二コーナー,第三コーナー,第四コーナー,東西,調教師,調教師ID,馬体重,馬体重(前走),増減
0,202405020801,1,8,18,ジェラルディーナ,2018105081,牝,4,56,Ｃ．デム,5473,133.0,0,4,8.1,11,12,11,9,西,斉藤崇史,1151,470,464,6
1,202405020801,2,7,13,ウインマリリン,2017101431,牝,5,56,レーン,5585,133.3,0.3,5,10.1,6,6,5,2,東,手塚貴久,1038,480,488,-8
2,202405020801,2,7,15,ライラック,2019103588,牝,3,54,Ｍ．デム,5212,133.3,0.0,12,52.9,16,17,14,13,東,相沢郁,1020,438,432,6
3,202405020801,4,7,14,アカイイト,2017106203,牝,5,56,幸英明,732,133.7,0.4,11,24.4,18,18,18,15,西,中竹和也,1039,524,528,-4
4,202405020801,5,6,11,ナミュール,2019105213,牝,3,54,横山武史,1170,133.7,0.0,3,7.3,8,10,8,11,西,高野友和,1117,448,446,2
5,202405020801,6,2,4,デアリングタクト,2017100720,牝,5,56,松山弘平,1126,134.0,0.3,1,4.3,8,8,8,9,西,杉山晴紀,1157,486,484,2
6,202405020801,7,3,6,ホウオウエミーズ,2017104691,牝,5,56,丸田恭介,1117,134.2,0.2,18,112.3,15,15,14,15,東,池上昌和,1150,444,448,-4
7,202405020801,8,6,12,ルビーカサブランカ,2017105582,牝,5,56,横山和生,1140,134.2,0.0,14,64.4,16,16,14,15,西,須貝尚介,1105,476,474,2
8,202405020801,9,2,3,ピンハイ,2019102772,牝,3,54,川田将雅,1088,134.5,0.3,6,11.8,6,6,5,6,西,田中克典,1180,412,414,-2
9,202405020801,10,4,7,イズジョーノキセキ,2017102294,牝,5,56,ルメール,5339,134.5,0.0,10,17.5,11,12,13,13,西,石坂公一,1166,470,466,4


In [7]:
paybackColName = ['raceId', '払戻種類', '一着', '二着', '三着', '払い戻し', '人気']
paybackDataFrame = pd.DataFrame(columns=paybackColName)
print(paybackDataFrame.columns)

Index(['raceId', '払戻種類', '一着', '二着', '三着', '払い戻し', '人気'], dtype='object')


In [8]:
response = requests.get(url)
bs = BeautifulSoup(response.content, 'html.parser')
html_string = str(bs)
html_io = StringIO(html_string)
tables = pd.read_html(html_io)
wide = tables[1]
tree = tables[2]
print(tree)

     0                          1                  2          3
0  ワイド    13 - 18 15 - 18 13 - 15  1,570 9,180 8,380   16 78 72
1   馬単            18 → 13 18 → 15       3,520 23,140     24 127
2  三連複               13 - 15 - 18              90210        231
3  三連単  18 → 13 → 15 18 → 15 → 13    206,260 289,250  1138 1391


In [20]:
raptime_table = tables[5]
raptime = raptime_table[1][0].split(' - ')
pace= raptime_table[1][1].replace('(',' - ').replace(')','').replace(' ','').split('-')
raptimeColName = ['raceId', 'name', '200m', '400m', '600m', '800m', '1000m', '1200m', '1400m', '1600m', '1800m', '2000m', '2200m', '2400m', '2600m', '2800m', '3000m', '3200m', '3400m', '3600m','first3f', 'last3f']
first3f = pace.pop(-2)
last3f = pace.pop(-1)
for i in range(len(pace)):
    print(pace[i])

35.3 36.4
12.6
23.5
35.3
47.9
60.3
72.6
84.8
96.6
108.5
120.8
133.0 


In [ ]:
for vertical in ['raptime', 'pace']:
    new_row['raceID'] = '202405020801'
    new_row['name'] = vertical
    for i in range(len(raptime)):
    

In [31]:
bet_type = ['単勝', '複勝', '枠連', '馬連', 'ワイド', '馬単', '三連複', '三連単']
paybackFrame = pd.DataFrame(columns=paybackColName)
for i in range(len(wide)):
    if wide.iloc[i][0]  == '単勝' or wide.iloc[i][0] == '複勝':
        bet = wide.iloc[i][0]
        hourse_num = wide.iloc[i][1].split(' ')
        payout = wide.iloc[i][2].split(' ')
        favorite = wide.iloc[i][3].split(' ')
        for j in range(len(hourse_num)):
            new_row = {
                'raceId': '202405020801',
                '払戻種類': bet,
                '一着': hourse_num[j],
                '二着': nanValue,
                '三着': nanValue,
                '払い戻し': payout[j].replace(',',''),
                '人気': favorite[j]
            }
            paybackFrame = pd.concat((paybackFrame,pd.DataFrame([new_row])), ignore_index=True)
    elif wide.iloc[i][0] == '枠連' or wide.iloc[i][0] == '馬連' or wide.iloc[i][0] == 'ワイド' or wide.iloc[i][0] == '馬単':
        bet = wide.iloc[i][0]
        hourse_num = wide.iloc[i][1].replace(' - ','-').split(' ')
        payout = wide.iloc[i][2].split(' ')
        favorite = wide.iloc[i][3].split(' ')
        for j in range(len(hourse_num)):
            tyakuzyun = hourse_num[j].split('-')
            new_row = {
                'raceId': '202405020801',
                '払戻種類': bet,
                '一着': tyakuzyun[0],
                '二着': tyakuzyun[1],
                '三着': nanValue,
                '払い戻し': payout[j].replace(',',''),
                '人気': favorite[j]
            }
            paybackFrame = pd.concat((paybackFrame,pd.DataFrame([new_row])), ignore_index=True)

for i in range(len(tree)):
    if tree.iloc[i][0] == '枠連' or tree.iloc[i][0] == '馬連' or tree.iloc[i][0] == 'ワイド' or tree.iloc[i][0] == '馬単':
        bet = tree.iloc[i][0]
        hourse_num = tree.iloc[i][1].replace(' - ','-').replace(' → ','-').split(' ')
        payout = tree.iloc[i][2].split(' ')
        favorite = tree.iloc[i][3].split(' ')
        for j in range(len(hourse_num)):
            tyakuzyun = hourse_num[j].split('-')
            new_row = {
                'raceId': '202405020801',
                '払戻種類': bet,
                '一着': tyakuzyun[0],
                '二着': tyakuzyun[1],
                '三着': nanValue,
                '払い戻し': payout[j].replace(',',''),
                '人気': favorite[j]
            }
            paybackFrame = pd.concat((paybackFrame,pd.DataFrame([new_row])), ignore_index=True)
    elif tree.iloc[i][0] == '三連複' or tree.iloc[i][0] == '三連単':
        bet = tree.iloc[i][0]
        hourse_num = tree.iloc[i][1].replace(' - ','-').replace(' → ','-').split(' ')
        payout = tree.iloc[i][2].split(' ')
        favorite = tree.iloc[i][3].split(' ')
        for j in range(len(hourse_num)):
            tyakuzyun = hourse_num[j].split('-')
            new_row = {
                'raceId': '202405020801',
                '払戻種類': bet,
                '一着': tyakuzyun[0],
                '二着': tyakuzyun[1],
                '三着': tyakuzyun[2],
                '払い戻し': payout[j].replace(',',''),
                '人気': favorite[j]
            }
            paybackFrame = pd.concat((paybackFrame,pd.DataFrame([new_row])), ignore_index=True)
print(paybackFrame)

          raceId 払戻種類  一着   二着   三着    払い戻し    人気
0   202405020801   単勝  18  NaN  NaN     810     4
1   202405020801   複勝  18  NaN  NaN     330     4
2   202405020801   複勝  13  NaN  NaN     370     6
3   202405020801   複勝  15  NaN  NaN    1160    13
4   202405020801   枠連   7    8  NaN    1410     7
5   202405020801   馬連  13   18  NaN    1920    13
6   202405020801   馬連  15   18  NaN   15500    70
7   202405020801  ワイド  13   18  NaN    1570    16
8   202405020801  ワイド  15   18  NaN    9180    78
9   202405020801  ワイド  13   15  NaN    8380    72
10  202405020801   馬単  18   13  NaN    3520    24
11  202405020801   馬単  18   15  NaN   23140   127
12  202405020801  三連複  13   15   18   90210   231
13  202405020801  三連単  18   13   15  206260  1138
14  202405020801  三連単  18   15   13  289250  1391


In [ ]:
#後で消す
def getTables(response):
    # HTTPステータスコードが200（成功）の場合のみ処理を続行
    if response.status_code == 200:
        # HTMLをパースしてBeautifulSoupオブジェクトを作成
        bs = BeautifulSoup(response.content, 'html.parser')
        bs = bs.decode('UTF-8')
        html_string = str(bs)
        html_io = StringIO(html_string)

        # テーブルデータを抽出
        tables = pd.read_html(html_io)
        df = tables[0]
        print(df)
        return df

    else:
        print(f"HTTPステータスコード {response.status_code}: ページの取得に失敗しました")
        

In [ ]:
#開催年
years = [str(i).zfill(4) for i in range(2021, 2022)]
#開催場所 01:札幌, 02:函館, 03:福島, 04:新潟, 05:東京, 06:中山, 07:中京, 08:京都, 09::阪神, 10:小倉
places = [str(i).zfill(2) for i in range (5, 6)]
#開催回
times = [str(i).zfill(2) for i in range(1, 2)]
#開催日
days = [str(i).zfill(2) for i in range(1, 2)]
#レースNo
races = [str(i).zfill(2) for i in range(1, 13)]

raceIdList = []
for y in years:
    for p in places:
        for t in times:
            for d in days:
                for r in races:
                    raceIdList.append(y + p + t + d + r)
                    

In [ ]:

url = 'https://db.netkeiba.com/race/'
colName = ['raceId', 'htmlBytes']
df = pd.DataFrame(columns=colName)
escapeList = []

# pickleファイルが存在するか確認し、データを読み込む
if os.path.isfile('race_html.pkl'):
    df = pd.read_pickle('race_html.pkl')
    escapeList = df['raceId'].to_list()

# ページネーション用の競走IDリストを生成する関数
def addEscapeList(id: str, ll: list):
    idAry = [id[0:4], id[4:6], id[6:8], id[8:10], id[10:12]]
    for r in range(1, 13):
        idAry[4] = str(r).zfill(2)
        ll.append(''.join(idAry))
    if idAry[3] == '01':
        for d in range(2, 9):
            idAry[3] = str(d).zfill(2)
            ll = addEscapeList(''.join(idAry), ll)
    if idAry[2] == '01':
        for t in range(2, 9):
            idAry[2] = str(t).zfill(2)
            ll = addEscapeList(''.join(idAry), ll)

    return ll

# raceIdListに実際の競走IDのリストがあると仮定します
# raceIdList = ...

for raceId in tqdm(raceIdList):
    try:
        if raceId in escapeList:
            continue
        response = url + raceId
        html = requests.get(response)
        #html.encoding = 'utf-8'
        soup = BeautifulSoup(html.content, 'html.parser')

        if 'レース結果' in soup.text:
            tmpDf = pd.DataFrame([[raceId, html.content]], columns=colName)  # 'colmuns'の誤りを修正
            df = pd.concat([df, tmpDf], axis=0, ignore_index=True)
        else:
            escapeList = addEscapeList(raceId, escapeList)  # 変数名を修正

        time.sleep(3)
    except Exception as e:  # 例外をキャッチしてログに記録
        print(f'例外が発生しました: {str(e)}')

# DataFrameをpickleファイルに保存
df.to_pickle('race_html.pkl')


In [ ]:
htmlDf = pd.read_pickle('race_html.pkl')

raceInfoList = []

#raceResultDf = pd.DataFrame()
for idx, dat in tqdm(htmlDf.iterrows(), total=len(htmlDf)):
    raceId = dat['raceId']
    htmlBytes = dat['htmlBytes']

    soup = BeautifulSoup(htmlBytes.decode('euc-jp','ignore'), 'html.parser')
    mainrace_data = soup.find('div', class_='mainrace_data')
    rowdata = {}
    rowdata['raceId'] = raceId
    rowdata['レース名'] = mainrace_data.find('h1').text
    rowdata['R'] = mainrace_data.find('dt').text.replace('\n','').replace(' ','').replace('R', '')
    spantexts = mainrace_data.find('span').text.replace('\xa0','').replace(' ','').split('/')
    
    rowdata['コース種'] = '障害' if '障' in spantexts[0] else 'ダート' if 'ダ' in spantexts[0] else '芝'
    
    rowdata['コース回り'] = '右' if '右' in spantexts[0] else '左' if '左' in spantexts[0] else '障害'

    rowdata['距離'] = int(re.findall('\d+',spantexts[0])[0])

    rowdata['天気'] = spantexts[1][3:]

    rowdata['馬場'] = spantexts[2].split(':')[1]
    rowdata['発走'] = spantexts[3][3:]

    smalltxt = mainrace_data.find('p', class_='smalltxt').text.replace('\xa0',' ').replace('  ',' ').split(' ')
    dt = datetime.datetime.strptime(smalltxt[0], '%Y年%m月%d日')
    rowdata['日付'] = dt.strftime('%Y/%m/%d')

    placeDict = {
        '01':'札幌', '02':'函館', '03':'福島', '04':'新潟', '05':'東京', '06':'中山', '07':'中京', '08':'京都', '09':'阪神', '10':'小倉'
    }
    rowdata['開催場所'] = placeDict[raceId[4:6]]

    if 'G1' in rowdata['レース名']:
        raceGrade = 'G1'
    elif 'G2' in rowdata['レース名']:
        raceGrade = 'G2'
    elif 'G3' in rowdata['レース名']:
        raceGrade = 'G3'
    elif '未勝利' in smalltxt[2]:
        raceGrade = '未勝利'
    elif '新馬' in smalltxt[2]:
        raceGrade = '新馬'
    elif '1勝クラス' in smalltxt[2] or '500万' in smalltxt[2]:
        raceGrade = '1勝クラス'
    elif '2勝' in smalltxt[2] or '1000万' in smalltxt[2]:
        raceGrade = '2勝クラス'
    elif '3勝' in smalltxt[2] or '1600万' in smalltxt[2]:   
        raceGrade = '3勝クラス'
    else:
        raceGrade = 'オープン'  

    if '牡・牝' in smalltxt[3]:
        restriction = '牡・牝'
    elif '牝' in smalltxt[3]:
        restriction = '牝'
    elif '牡' in smalltxt[3]:
        restriction = '牡'
    else:
        restriction = '無'

    rowdata['制限'] = restriction

    if 'ハンデ' in smalltxt[3]:
        handicap = 'ハンデ'
    elif '別定' in smalltxt[3]:
        handicap = '別定'
    else:
        handicap = '定量'
    
    rowdata['ハンデ'] = handicap

    raceInfoList.append(rowdata)
raceInfoDf = pd.DataFrame(raceInfoList)
raceInfoDf.to_pickle('race_info.pkl')

In [ ]:
table = soup.find_all('table')[0]

columns = []

for head in table.find_all('th'):
    columns.append(head.text)

columns = ['raceId'] + columns + ['horseId', 'jockeyId', 'trainerId']
df = pd.DataFrame(columns=columns)

for i, row in enumerate(table.find_all('tr')):
    if i == 0:
        continue
    items = [raceId]

    cells = row.find_all('td')

    for cell in cells:
        items.append(cell.text.replace('\n',''))

    items.append(str(cells[3]).split('/horse/')[1].split('/')[0])
    items.append(str(cells[6]).split('/recent/')[1].split('/')[0])
    items.append(str(cells[18]).split('/recent')[1].split('/')[0])

    df.loc[i] = items
    break

raceResultDf = pd.concat([raceResultDf, df],axis=0)


In [ ]:
df.to_csv('data.csv',encoding='utf-8')
df.columns
print(df)